In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key= os.getenv('GOOGLE_API_KEY')
if not api_key :
    raise (' Not key')

In [3]:
from langchain_core.documents import Document

# Đây là danh sách 3 văn bản dài, chưa được cắt gọt
raw_contents = [
    # VĂN BẢN 1: Tin tức Lãi suất (Thử thách: Nhiều con số % dễ gây nhầm lẫn)
    """
    Tieu diem lai suat ngan hang thang 2/2026: Xu huong giam cham lai, xuat hien su phan hoa.
    Theo khao sat tai 30 ngan hang thuong mai trong nuoc sang nay, bieu lai suat huy dong da co su dieu chinh trai chieu.
    Cu the, Ngan hang TMCP Sai Gon Thuong Tin (Sacombank) vua thong bao giam 0,2 diem % o cac ky han duoi 6 thang.
    Theo do, lai suat ky han 1 thang con 2,8%/nam, ky han 3 thang con 3,1%/nam.
    Trai nguoc voi xu huong nay, Ngan hang VIB lai bat ngo tang nhe lai suat ky han 12 thang them 0,1 diem %, len muc 5,4%/nam de thu hut dong von trung han.
    Doi voi nhom Big4 (Vietcombank, BIDV, Agribank, VietinBank), lai suat van duy tri o muc thap ky luc.
    Vietcombank niem yet ky han 12 thang o muc 4,6%/nam, thap nhat he thong.
    Dang chu y, cac chuyen gia du bao lai suat se kho giam sau them nua do ap luc ty gia USD/VND dang tang tro lai trong boi canh Fed chua ha lai suat.
    (Nguon: CafeF - Nhip song kinh te).
    """,

    # VĂN BẢN 2: Báo cáo Tài chính HPG (Thử thách: So sánh năm này qua năm khác)
    """
    Tap doan Hoa Phat (HPG) cong bo ket qua kinh doanh Quy 4/2025 voi nhung con so an tuong.
    Doanh thu thuan dat 35.000 ty dong, tang truong 25% so voi cung ky nam 2024.
    Luy ke ca nam 2025, Hoa Phat ghi nhan doanh thu hop nhat dat 150.000 ty dong.
    Tuy nhien, loi nhuan sau thue lai ghi nhan muc giam nhe 5%, ve muc 8.500 ty dong.
    Nguyen nhan chu yeu den tu viec chi phi dau vao (quang sat, than coc) tang cao trong nua cuoi nam.
    Ve san luong, thep xay dung tiep tuc la chu luc khi ban ra 4,5 trieu tan, tang 12% so voi nam truoc.
    Mang nong nghiep dong gop 5% vao tong loi nhuan, tuong duong 425 ty dong.
    Ban lanh dao Tap doan dat muc tieu nam 2026 se phuc hoi bien loi nhuan gop len muc 15% nho viec dua vao van hanh lo cao Dung Quat 2.
    """,

    # VĂN BẢN 3: Thị trường Vàng (Thử thách: Biến động giá liên tục, nhiều mốc thời gian)
    """
    Sang ngay 2/2/2026, gia vang trong nuoc tiep tuc "nhay mua" theo da tang cua the gioi.
    Mo cua phien sang, Cong ty Vang bac Da quy Sai Gon (SJC) niem yet gia vang mieng o muc 85,5 trieu dong/luong (mua vao) va 88,5 trieu dong/luong (ban ra).
    Chenh lech mua - ban duoc noi rong len toi 3 trieu dong/luong, day rui ro ve phia nguoi mua.
    Tai Bao Tin Minh Chau, gia vang nhan tron tron cung lap dinh moi lich su tai 78 trieu dong/luong.
    Den phien chieu cung ngay, ap luc chot loi xuat hien khien gia vang mieng SJC quay dau giam 500.000 dong/luong, xuong con 88 trieu dong/luong chieu ban ra.
    Gioi phan tich canh bao nha dau tu can can trong khi mua vao tai vung gia dinh, dac biet khi chi so DXY (suc manh dong USD) dang co dau hieu phuc hoi tu muc 102 diem.
    """
]

# Chuyển đổi thành dạng Document của LangChain
documents = [Document(page_content=text, metadata={"source": f"doc_{i+1}"}) for i, text in enumerate(raw_contents)]

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
model='BAAI/bge-m3'
embeddings=HuggingFaceEmbeddings(model_name=model)

2026-02-02 00:16:01.245622: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-02 00:16:01.282954: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-02 00:16:02.528134: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/chinghia/miniconda3/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: F

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma.vectorstores import Chroma

doc=documents

split=RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 100,
)
doc_split= split.split_documents(doc)

if os.path.exists('./chroma_cosine'):
    chroma=Chroma(
        embedding_function=embeddings,
        persist_directory='./chroma_cosine'
    )
else:
    chroma=Chroma.from_documents(
        doc_split,
        embedding=embeddings,
        persist_directory='./chroma_cosine'
    )
retriever_cosine=chroma.as_retriever(search_kwargs={'k':10})

In [8]:
import numpy as np
from scipy.spatial.distance import cdist

def energy_base_distance(X,Y):
    
    X=np.asarray(X)
    Y=np.asarray(Y)
#Tuong tac cheo
    d_xy=cdist(X,Y,metric='euclidean')
    E_xy=np.mean(d_xy)
#Tu nang x
    d_xx=cdist(X,X,metric='euclidean')
    E_xx=np.mean(d_xx)
#Tu nang y
    d_yy=cdist(Y,Y,metric='euclidean')
    E_yy=np.mean(d_yy)
#Energy_distance
    ED=2*E_xy - E_xx - E_yy

    return max(0,ED)

In [9]:
X = [[1,1],[2,3]]
Y = [[1,1],[2,3]]

ed=energy_base_distance(X,Y)
print(f'{ed}')

0


In [10]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_with_energy_clustering(query, retriever, embeddings, k_cluters=3):

    docs=retriever.invoke(query)

    context=[doc.page_content for doc in docs]

    doc_vector= np.array(embeddings.embed_documents(context))
    query_vector=np.array(embeddings.embed_query(query)).reshape(1,-1)
#kiem loi
    sims = cosine_similarity(query_vector, doc_vector)[0]
    print("Max cosine:", np.max(sims))
    if np.max(sims) < 0.35:
        return [] 
#tao cum
    actual_k=min(k_cluters,len(doc_vector))
    kmeans=KMeans(n_clusters= actual_k, random_state=42, n_init='auto')
    labels=kmeans.fit_predict(doc_vector)
#kiem tra e
    best_energy=float('inf')
    best_cluster_idx=-1
    for i in range(actual_k):
        indices=np.where(labels==i)[0]
        cluster_vector=doc_vector[indices]

        energy=energy_base_distance(query_vector,cluster_vector)

        if energy < best_energy:
            best_energy=energy
            best_cluster_idx=i
#tim idx e thap
    win_index=np.where(labels==best_cluster_idx)[0]
    print(f'{best_energy}')
#doan context e thap
    final_docs=[context[i] for i in win_index]

    return final_docs

In [11]:
query='Lợi nhuận năm 2025 của Hòa Phát tăng hay giảm so với năm trước?'
test=retrieve_with_energy_clustering(query,retriever_cosine,embeddings)
print(f'{test}')

Max cosine: 0.5582880394870501
1.6274768718699546
['Tap doan Hoa Phat (HPG) cong bo ket qua kinh doanh Quy 4/2025 voi nhung con so an tuong.\n    Doanh thu thuan dat 35.000 ty dong, tang truong 25% so voi cung ky nam 2024.\n    Luy ke ca nam 2025, Hoa Phat ghi nhan doanh thu hop nhat dat 150.000 ty dong.\n    Tuy nhien, loi nhuan sau thue lai ghi nhan muc giam nhe 5%, ve muc 8.500 ty dong.\n    Nguyen nhan chu yeu den tu viec chi phi dau vao (quang sat, than coc) tang cao trong nua cuoi nam.', 'Ve san luong, thep xay dung tiep tuc la chu luc khi ban ra 4,5 trieu tan, tang 12% so voi nam truoc.\n    Mang nong nghiep dong gop 5% vao tong loi nhuan, tuong duong 425 ty dong.\n    Ban lanh dao Tap doan dat muc tieu nam 2026 se phuc hoi bien loi nhuan gop len muc 15% nho viec dua vao van hanh lo cao Dung Quat 2.']
